## Install dependencies

In [3]:
import mediapipe as mp
import cv2 as cv


In [4]:
mp_drawing = mp.solutions.drawing_utils
mp_holistic = mp.solutions.holistic

## Get Webcam feed

In [9]:
# Connect to cam
cap = cv.VideoCapture(0)

with mp_holistic.Holistic( 
    min_detection_confidence=0.5, 
    min_tracking_confidence=0.5
) as holistic:
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        # Show image
        frame = cv.flip(frame, 1)
        image = cv.cvtColor(frame, cv.COLOR_BGR2RGB)

        # Make detections
        results = holistic.process(image)
        print(results)

        cv.imshow('Webcam', frame)

        # check if any keys are being pressed, if this is 'q' break out of while loop and release webcam & close frame
        if cv.waitKey(1) & 0xFF == ord('q'):
            break
cap.release()
cv.destroyAllWindows()

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

## Apply Styling